In [91]:
import dice

In [92]:
def gen_results_dict(sides=6):
    results = dict()
    for x in range(1,sides+1):
        results[x]=0
    return results

In [93]:
def roll_x_dice(num, sides=6):
    if not isinstance(sides, int):
        raise Exception('Sides must be integer')
    if not isinstance(num, int):
        raise Exception('Num must be integer!')
    res = dice.roll(str(num)+'d'+str(sides))
    return res

In [94]:
def get_results(rolls, requirement, sides=6):
    successes = 0
    fails = 0
    results = gen_results_dict(sides)
    for result in rolls:
        results[result] += 1
        if result >= requirement:
            successes += 1
        else:
            fails += 1
    return {"successes": successes, "fails": fails, "individual_results": results}

In [95]:
res = roll_x_dice(100)
final=get_results(res, 4)
print(final)


{'successes': 60, 'fails': 40, 'individual_results': {1: 7, 2: 17, 3: 16, 4: 22, 5: 21, 6: 17}}


In [96]:
def reroll_results(original_results, requirement, sides=6):
    original_fails = 0
    final_successes = 0
    final_fails = 0
    results = gen_results_dict(sides)
    
    for result in original_results:
        if result >= requirement:
            final_successes += 1
            results[result] += 1
        else:
            original_fails +=1
            
    rerolls = roll_x_dice(original_fails, sides)
    for result in rerolls:
        results[result] += 1
        if result >= requirement:
            final_successes += 1
        else:
            final_fails += 1
    return {"successes": final_successes, "fails": final_fails, "individual_results": results}
    
    
    

In [97]:
def reroll_ones(original_results, requirement, sides=6):
    ones = 0
    final_successes = 0
    final_fails = 0
    results = gen_results_dict(sides)
    
    for result in original_results:
        if result >= requirement:
            final_successes += 1
            results[result] += 1
        elif result == 1:
            ones +=1
        else:
            results[result] += 1
            final_fails +=1
            
    rerolls = roll_x_dice(ones, sides)
    for result in rerolls:
        results[result] += 1
        if result >= requirement:
            final_successes += 1
        else:
            final_fails += 1
    return {"successes": final_successes, "fails": final_fails, "individual_results": results}

In [98]:
res = roll_x_dice(10000)
final=reroll_ones(res, 4)
print(final)


{'successes': 5784, 'fails': 4216, 'individual_results': {1: 284, 2: 1882, 3: 2050, 4: 1857, 5: 1965, 6: 1962}}


In [99]:
sample_specs = {
    "bs":3,
    "shots":"3",
    "s":5,
    "ap":1,
    "d":"2",
    "reroll_hit":"ones", #"ones", "all", "none"
    "reroll_wound":"ones", #"ones", "all", "none"
    "lightning_fast":True # True, False
}

In [100]:
def parse_dice_string(dice_string):
    output = {
        "type_string":'',
        "number_of_dice":1
    }
    d_index = dice_string.index('d')
    type_string = 'd'+dice_string[d_index+1]
    output["type_string"] = type_string
    if d_index != 0:
        number_of_dice = dice_string[0]
        output["number_of_dice"]=number_of_dice
    return output
        

In [101]:
print(parse_dice_string('3d3'))

{'type_string': 'd3', 'number_of_dice': '3'}


In [102]:
def determine_number_of_shots(specs):
    shots = specs["shots"]
    if "d" in shots:
        parsed_dice_string = parse_dice_string(shots)
        dice_result = dice.roll(parsed_dice_string['number_of_dice']+parsed_dice_string['type_string'])
        return sum(dice_result)
    else:
        return int(shots)

In [103]:
sample_specs = {
    "bs":4,
    "shots":"1000",
    "s":5,
    "ap":1,
    "d":"2",
    "reroll_hit":"none", #"ones", "all", "none"
    "reroll_wound":"none", #"ones", "all", "none"
    "lightning_fast":True # True, False
}

determine_number_of_shots(sample_specs)

1000

In [104]:
def normalise_hit_requirement(req):
    if req>6:
        return 6
    elif req<2:
        return 2
    else:
        return req

def roll_to_hit(specs):
    number_of_shots = determine_number_of_shots(specs)
    hit_rolls = roll_x_dice(number_of_shots)
    hit_requirement = normalise_hit_requirement(specs["bs"]-1 if specs["lightning_fast"] else specs["bs"] )
    print(hit_requirement)
    if specs["reroll_hit"]=="ones":
        return reroll_ones(hit_rolls, specs["bs"])
    elif specs["reroll_hit"]=="all":
        return reroll_results(hit_rolls, specs["bs"])
    else:
        return get_results(hit_rolls,specs["bs"] )

In [105]:
hit_results = roll_to_hit(sample_specs)

3


In [106]:
def get_wound_requirements(strength):
    if strength == 6:
        return 4
    elif strength < 6:
        if strength < 4:
            return 6
        else:
            return 5
    else :
        if strength > 11:
            return 2
        else:
            return 3
        

In [107]:
def roll_to_wound(specs, hit_results):
    wound_requirement = get_wound_requirements(specs["s"])
    wound_rolls = roll_x_dice(hit_results['successes'])
    if specs["reroll_wound"]=="ones":
        return reroll_ones(wound_rolls, wound_requirement)
    elif specs["reroll_wound"]=="all":
        return reroll_results(wound_rolls,wound_requirement)
    else:
        return get_results(wound_rolls,wound_requirement)

In [108]:
roll_to_wound(sample_specs,hit_results)

{'successes': 165,
 'fails': 345,
 'individual_results': {1: 81, 2: 84, 3: 86, 4: 94, 5: 78, 6: 87}}

In [ ]:
def get_save_requirement(specs):
    pass